In [1]:
# sys module
import os
import sys
import nltk
from sklearn.model_selection import train_test_split 
from nltk.corpus import stopwords
from nltk.classify import naivebayes
import string

# third parties module
import pandas as pd

# local module
from data import load_data_sentiment, load_lexicon_sentiment

In [16]:
data = load_data_sentiment ()

In [17]:
# split train, val, test
train, test = train_test_split (data)
train = train.reset_index (drop=True)
test = test.reset_index (drop=True)
train, valid = train_test_split (train)
train = train.reset_index (drop=True)
valid = valid.reset_index (drop=True)
print (train.shape)
print (test.shape)
print (valid.shape)
# introduction to K-Fold
# confusion Matrix, F1-Metrics

(420, 2)
(187, 2)
(141, 2)


In [18]:
stopword = stopwords.words ('english')
def preprocessing (comment):
    # remove punctuation
    #for p in string.punctuation:
    #    comment = comment.replace (p, ' ')  
    # tokenize
    words = set (nltk.word_tokenize (comment.lower ()))
    # remove char less than 3
    words = set ([w for w in words if len (w) > 3])  
    # remove stopwords
    words = words - set (stopword)
    
    return words

# feature extraction
def extract_feature (word):
    feature = {}
    for w in word:
        feature[w] = True
    return feature

In [19]:
train['word'] = train['comment'].apply (preprocessing)
train['feature'] = train['word'].apply (extract_feature)

In [20]:
classifier = naivebayes.NaiveBayesClassifier.train (train[['feature', 'sentiment']].values)
classifier.show_most_informative_features (10)

Most Informative Features
                    plot = True                0 : 1      =      5.2 : 1.0
                terrible = True                0 : 1      =      4.8 : 1.0
                 nothing = True                0 : 1      =      4.8 : 1.0
                    show = True                0 : 1      =      4.8 : 1.0
                    make = True                0 : 1      =      4.8 : 1.0
               excellent = True                1 : 0      =      4.5 : 1.0
                   liked = True                1 : 0      =      4.5 : 1.0
                    love = True                1 : 0      =      4.4 : 1.0
                  things = True                0 : 1      =      4.1 : 1.0
                   would = True                0 : 1      =      4.0 : 1.0


In [21]:
valid['word'] = valid['comment'].apply (preprocessing)
valid['feature'] = valid['word'].apply (extract_feature)

In [22]:
# predict
valid['prediction'] = valid['feature'].apply (classifier.classify)
# show between target and prediction
valid['is_true'] = (valid['sentiment'] == valid['prediction']).astype (int)
accuracy = sum (valid['is_true']) / valid.shape[0]
print ("Accuracy : {:.2F}".format (accuracy))

Accuracy : 0.79


In [23]:
# After fine tuned the parameter use test data only ONCE
test['word'] = test['comment'].apply (preprocessing)
test['feature'] = test['word'].apply (extract_feature)

In [24]:
# predict
test['prediction'] = test['feature'].apply (classifier.classify)
# show between target and prediction
test['is_true'] = (test['sentiment'] == test['prediction']).astype (int)
accuracy = sum (test['is_true']) / test.shape[0]
print ("Accuracy : {:.2F}".format (accuracy))

Accuracy : 0.76


In [25]:
# using KFold
from sklearn.model_selection import KFold, StratifiedKFold

kfold = StratifiedKFold (n_splits=5, shuffle=True)

folds = kfold.split (data, data['sentiment'])
for train_idx, valid_idx in folds:
    ftrain  = data.loc[train_idx]
    ftrain = train.reset_index (drop=True)
    
    fvalid = data.loc[valid_idx]
    fvalid = valid.reset_index (drop=True)
    
    # training with model
    # test with valid
# average

In [26]:
from sklearn.metrics import confusion_matrix, f1_score
print (confusion_matrix (valid['sentiment'], valid['prediction'], labels=[0, 1]))

print (f1_score (valid['sentiment'], valid['prediction'], pos_label=0))

[[56 11]
 [18 56]]
0.7943262411347517
